In [1]:
import pickle
import torch
import pandas as pd
from torch_geometric.data import Data,DataLoader
from functions import *
from pytorch_util import *
from torch.optim import Adam
from torch.nn.utils import clip_grad_value_
from torch_geometric.nn import NNConv,GATConv

In [2]:
with open('../Data/train_data_ACSF.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('../Data/val_data_ACSF.pickle', 'rb') as handle:
    val_data = pickle.load(handle)
with open('../Data/test_data_ACSF.pickle', 'rb') as handle:
    test_data = pickle.load(handle)

In [3]:
### parameters ###
batch_size = 32
dim = 64
edge_dim = 12
epochs = 5
clip = 2

### load dataset

In [4]:
train_list = [Data(**d) for d in train_data]
train_dl = DataLoader(train_list,batch_size,shuffle=True)

In [5]:
val_list = [Data(**d) for d in val_data]
valid_dl = DataLoader(val_list,batch_size,shuffle=False)

In [6]:
test_list = [Data(**d) for d in test_data]
test_dl = DataLoader(test_list,batch_size,shuffle=False)

In [17]:
class Net_int_2Edges_attention(torch.nn.Module):
    # use both types of edges
    def __init__(self,dim=64,edge_dim=12,node_in=8,edge_in=19,edge_in3=8):
        super(Net_int_2Edges_attention, self).__init__()
        self.lin_node = torch.nn.Linear(node_in, dim)
        
        self.conv1 = GATConv(dim, dim, negative_slope=0.2, dropout=0.1, bias=True)
        self.gru1 = GRU(dim, dim)
        self.lin_covert = Sequential(BatchNorm1d(dim),Linear(dim, dim*2), \
                                     RReLU(), Dropout(),Linear(dim*2, dim*2),RReLU())
        
        self.conv2 = GATConv(dim*2, dim*2, negative_slope=0.2, dropout=0.1, bias=True)
        self.gru2 = GRU(dim*2, dim*2)
        
        self.lin_weight = Linear(8, dim*3*2, bias=False)
        self.lin_bias = Linear(8, 1, bias=False)
        self.norm = BatchNorm1d(dim*3*2)
        self.norm_x = BatchNorm1d(node_in)
        
    def forward(self, data,IsTrain=False):
        out = F.rrelu(self.lin_node(self.norm_x(data.x)))
        h = out.unsqueeze(0)
        # edge_*3 only does not repeat for undirected graph. Hence need to add (j,i) to (i,j) in edges
        edge_index3 = torch.cat([data.edge_index3,data.edge_index3[[1,0]]],1)
        
        for i in range(2):
            # using bonding as edge
            m = F.rrelu(self.conv1(out, data.edge_index))
            out, h = self.gru1(m.unsqueeze(0), h)
            out = out.squeeze(0)
        
        out = self.lin_covert(out)
        h = out.unsqueeze(0)
        for i in range(2):
            # using couping as edge
            m = F.rrelu(self.conv2(out, edge_index3))
            out, h = self.gru2(m.unsqueeze(0), h)
            out = out.squeeze(0)     
            
        temp = out[data.edge_index3] # (2,N,d)
        yhat = torch.cat([temp.mean(0),temp[0]*temp[1],(temp[0]-temp[1])**2],1)
        yhat = self.norm(yhat)
        weight = self.lin_weight(data.edge_attr3)
        bias = self.lin_bias(data.edge_attr3)
        yhat = torch.sum(yhat * weight,1,keepdim=True) + bias
        yhat = yhat.squeeze(1)
        
        if IsTrain:
            k = torch.sum(data.edge_attr3,0)
            nonzeroIndex = torch.nonzero(k).squeeze(1)
            abs_ = torch.abs(data.y-yhat).unsqueeze(1)
            loss = torch.sum(torch.log(torch.sum(abs_ * data.edge_attr3[:,nonzeroIndex],0)/k[nonzeroIndex]))/nonzeroIndex.shape[0]
            return loss
        else:
            return yhat            

### build model and set up training

In [18]:
model = Net_int_2Edges_attention(dim=dim,edge_dim=edge_dim,node_in=89).to('cuda:0')

In [19]:
paras = trainable_parameter(model)
opt = Adam(paras,lr=1e-4)

In [20]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:1.2763230401084038, val_loss:0.9297485272599082
epoch:1, train_loss:0.8827104350448909, val_loss:0.5829253540589259
epoch:2, train_loss:0.5751959482692506, val_loss:0.360000357318383
epoch:3, train_loss:0.4788674449724425, val_loss:0.2879477939926661
epoch:4, train_loss:0.4252034093985342, val_loss:0.26048794730256003
Training completed in 183.16261506080627s


In [21]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.3848493720469106, val_loss:0.1993090063970313
epoch:1, train_loss:0.3507571903316736, val_loss:0.17040536584507707
epoch:2, train_loss:0.32255915740090085, val_loss:0.1436848601835788
epoch:3, train_loss:0.2991853760028191, val_loss:0.13501086126630887
epoch:4, train_loss:0.27735012987731633, val_loss:0.1102342944090756
Training completed in 184.1844780445099s


In [22]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.25745262629060806, val_loss:0.13251528419299513
epoch:1, train_loss:0.23986363048640952, val_loss:0.06158646635918154
epoch:2, train_loss:0.21997781106065167, val_loss:0.05651860962359187
epoch:3, train_loss:0.2020271917507836, val_loss:0.0556906516241849
epoch:4, train_loss:0.18856826045256148, val_loss:0.039772372553523026
Training completed in 184.19097995758057s


In [23]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.1730741000115517, val_loss:-0.005460996109132583
epoch:1, train_loss:0.15753427020026686, val_loss:0.007988813062572582
epoch:2, train_loss:0.14526036396838987, val_loss:-0.015442951180391077
epoch:3, train_loss:0.13380482115250064, val_loss:-0.04639755710004232
epoch:4, train_loss:0.12101289225724335, val_loss:-0.04475328815368633
Training completed in 183.52749705314636s


In [24]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.10756200827593448, val_loss:-0.07759536872020899
epoch:1, train_loss:0.09572975385455465, val_loss:-0.08480762765925919
epoch:2, train_loss:0.08617210714465878, val_loss:-0.09768415895155352
epoch:3, train_loss:0.07523118277690592, val_loss:-0.11751206976990415
epoch:4, train_loss:0.0667623251243901, val_loss:-0.0751465323787087
Training completed in 183.50308871269226s


In [25]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.056539723785891345, val_loss:-0.1293246566564736
epoch:1, train_loss:0.04672595400705586, val_loss:-0.13367784428450033
epoch:2, train_loss:0.03699686924684631, val_loss:-0.1411614723933431
epoch:3, train_loss:0.029545827227494298, val_loss:-0.14470523062488463
epoch:4, train_loss:0.020884120063519827, val_loss:-0.14485183558784998
Training completed in 183.04349184036255s


In [26]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:0.012147115631208715, val_loss:-0.15060853430380425
epoch:1, train_loss:0.00547886295921191, val_loss:-0.19225528692173907
epoch:2, train_loss:-0.004953912531281641, val_loss:-0.1972732374794845
epoch:3, train_loss:-0.011103451136153858, val_loss:-0.1912434179431353
epoch:4, train_loss:-0.019053406872887507, val_loss:-0.2060453997542843
Training completed in 184.27624654769897s


In [27]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.02488942944351456, val_loss:-0.21592929436323735
epoch:1, train_loss:-0.032008174093969076, val_loss:-0.22090186309228596
epoch:2, train_loss:-0.03953355045969567, val_loss:-0.23007104389019248
epoch:3, train_loss:-0.04506736539498442, val_loss:-0.22814361333178404
epoch:4, train_loss:-0.05032463061823848, val_loss:-0.24515417845458046
Training completed in 183.63969111442566s


In [28]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.056076685075699656, val_loss:-0.21191699623775023
epoch:1, train_loss:-0.06432654899232018, val_loss:-0.26913527427957606
epoch:2, train_loss:-0.06943999793828487, val_loss:-0.2661238138953972
epoch:3, train_loss:-0.07637670851313855, val_loss:-0.24071207164157915
epoch:4, train_loss:-0.08116339042333091, val_loss:-0.2532991055224059
Training completed in 184.8323655128479s


In [29]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.08460871098971035, val_loss:-0.23330696708021256
epoch:1, train_loss:-0.09024701279063829, val_loss:-0.270684179252921
epoch:2, train_loss:-0.09568540197313785, val_loss:-0.27377040278262055
epoch:3, train_loss:-0.1004695739677141, val_loss:-0.3003609601376403
epoch:4, train_loss:-0.10500206800359103, val_loss:-0.3095818685893065
Training completed in 185.3657205104828s


In [30]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.11025868050641648, val_loss:-0.2983772781255663
epoch:1, train_loss:-0.11561562873126711, val_loss:-0.28712187588023835
epoch:2, train_loss:-0.1179268138136529, val_loss:-0.2967251596462905
epoch:3, train_loss:-0.12374504313390713, val_loss:-0.310354254152785
epoch:4, train_loss:-0.12778081925211973, val_loss:-0.31696667598608214
Training completed in 183.35422682762146s


In [31]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.1314972408889198, val_loss:-0.2951288130452745
epoch:1, train_loss:-0.13562149750209312, val_loss:-0.3300426109479024
epoch:2, train_loss:-0.13980778764250293, val_loss:-0.3406190392043855
epoch:3, train_loss:-0.1435997973178749, val_loss:-0.3294810190414771
epoch:4, train_loss:-0.14497090507831867, val_loss:-0.3271567957141461
Training completed in 183.31360912322998s


In [32]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.1516059122419229, val_loss:-0.34702933727739715
epoch:1, train_loss:-0.1542879439793292, val_loss:-0.33814862513771426
epoch:2, train_loss:-0.15827068540250186, val_loss:-0.3545774796929879
epoch:3, train_loss:-0.16347309937474933, val_loss:-0.3565521120197243
epoch:4, train_loss:-0.16565959682755402, val_loss:-0.3614037340650192
Training completed in 183.5092658996582s


In [33]:
since = time.time()
opt.zero_grad()
for epoch in range(epochs):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.1701038785987403, val_loss:-0.361023797852616
epoch:1, train_loss:-0.17207405546013219, val_loss:-0.36406097319136316
epoch:2, train_loss:-0.17655886846130767, val_loss:-0.35708709717051595
epoch:3, train_loss:-0.18011566314371033, val_loss:-0.3700349324534082
epoch:4, train_loss:-0.18109996554326346, val_loss:-0.37857172902450603
Training completed in 183.14181089401245s


In [34]:
since = time.time()
opt.zero_grad()
for epoch in range(10):
    # training #
    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    for i,data in enumerate(train_dl):
        data = data.to('cuda:0')
        loss = model(data,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()

    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(valid_dl):
            data = data.to('cuda:0')
            loss = model(data,True)
            val_loss += loss.item()
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))
    
time_elapsed = time.time() - since
print('Training completed in {}s'.format(time_elapsed))

epoch:0, train_loss:-0.18509672055123982, val_loss:-0.38541898357435167
epoch:1, train_loss:-0.18923198880741027, val_loss:-0.37344255164647716
epoch:2, train_loss:-0.19087553226312765, val_loss:-0.37672733935790187
epoch:3, train_loss:-0.19538171977915336, val_loss:-0.36737367634971935
epoch:4, train_loss:-0.19756918384514283, val_loss:-0.3818835368714271
epoch:5, train_loss:-0.20110283556611885, val_loss:-0.3816475563197054
epoch:6, train_loss:-0.20399338434649197, val_loss:-0.39975956881530267
epoch:7, train_loss:-0.20721842018631043, val_loss:-0.406651510260044
epoch:8, train_loss:-0.20886401680332642, val_loss:-0.40507451546752554
epoch:9, train_loss:-0.21081084837931774, val_loss:-0.4007577798369094
Training completed in 368.64795184135437s


In [45]:
checkpoint = torch.load('../Model/gnn_int_logloss_2Edges.tar')
model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])

In [109]:
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            }, '../Model/gnn_int_logloss_2Edges_ACSF.tar')

### make submissions

In [110]:
model.eval()
yhat_list = []
with torch.no_grad():
    for data in test_dl:
        data = data.to('cuda:0')
        yhat_list.append(model(data,False))

yhat = torch.cat(yhat_list).cpu().detach().numpy()
submission = pd.read_csv('../Data/sample_submission.csv')
submission['scalar_coupling_constant'] = yhat
submission.to_csv('../Submission/gnn_int_logloss_2Edges_ACSF.csv',index=False)